In [1]:
import numpy as np
import pandas as pd

# pd.set_option('display.height', 1000)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.set_option("max_colwidth", 200)
from IPython.core.display import HTML, display

display(HTML("<style>.container { width:100% !important; }</style>"))


def df_stats(df):
    from tabulate import tabulate

    print("\n***** Shape: ", df.shape, " *****\n")

    columns_list = df.columns.values.tolist()
    isnull_list = df.isnull().sum().values.tolist()
    isunique_list = df.nunique().values.tolist()
    dtypes_list = df.dtypes.tolist()

    list_stat_val = list(zip(columns_list, isnull_list, isunique_list, dtypes_list))
    df_stat_val = pd.DataFrame(list_stat_val, columns=["Name", "Null", "Unique", "Dtypes"])
    print(tabulate(df_stat_val, headers="keys", tablefmt="psql"))
    return df.head()

/tmp/ipykernel_56001/1374276192.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [ ]:
import pickle

# Load the pickle file containing (all_reports, all_text_embeddings)
with open("text_embeddings.pkl", "rb") as f:
    all_reports, all_text_embeddings = pickle.load(f)

# Print the first 5 indices and their corresponding reports
for i in range(5):
    print(f"Index {i}: {all_reports[i]}")

In [ ]:
print(len(all_reports))

In [5]:
# Load the CSV file into a DataFrame
val_epoch10_df = pd.read_csv("outputs/deepCORO_CLIP/DeepCORO_Clip_Sweep_Learnable_Temp_Full_mvit_b12_f16_RAdam_lr1.1973658238534876e-05_20250103-093613_evid9bon/val_epoch10.csv")
# Load the CSV file with alpha character separator into a DataFrame
reports_sampled_df = pd.read_csv("data/reports/reports_sampled_no_conclusion.csv", sep='α')


/tmp/ipykernel_56001/2153714562.py:4: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  reports_sampled_df = pd.read_csv("data/reports/reports_sampled_no_conclusion.csv", sep='α')


In [ ]:
### Stenting procedure

In [7]:
# Filter reports_sampled_df by Split == 'val'
val_reports_sampled_df = reports_sampled_df[reports_sampled_df['Split'] == 'val'].reset_index()

   index  Unnamed: 0.1  Unnamed: 0                                                                                                                                               filename                                                                                                                                                                       y_hat  object_value brand   sex  FPS  NumberOfFrames      date  study_time  series_time   birthdate color_format                                            StudyID                                StudyInstanceUID                                  SeriesInstanceUID                                                                                                                                                                    dicom_path                                                                                                                                               FileName  uint16_video  primary_angle  secondary_angle  width  \
0   

In [19]:
display(val_reports_sampled_df.StudyInstanceUID.nunique())
display(val_epoch10_df.ground_truth_idx.max())
display(val_epoch10_df.predicted_idx_4.max())

3463

np.int64(3462)

np.int64(2488)